From https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
!tar xvzf news20.tar.gz

In [1]:
import os

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
GLOVE_DIR = os.path.join(DATA_DIR, 'glove/glove.6B')
TEXT_DATA_DIR = os.path.join(DATA_DIR, '20_newsgroup')

In [2]:
import keras.preprocessing.text
import string

# Workaround to add "Unicode support for keras.preprocessing.text"
# (https://github.com/fchollet/keras/issues/1072#issuecomment-295470970)
def text_to_word_sequence(text,
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True, split=" "):
    if lower: text = text.lower()
    if type(text) == unicode:
        translate_table = {ord(c): ord(t) for c,t in zip(filters, split*len(filters)) }
    else:
        translate_table = string.maketrans(filters, split * len(filters))
    text = text.translate(translate_table)
    seq = text.split(split)
    return [i for i in seq if i]
    
keras.preprocessing.text.text_to_word_sequence = text_to_word_sequence

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).

GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)

20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

from __future__ import print_function

import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))# second, prepare text samples and their labels

Indexing word vectors.
Found 400000 word vectors.


In [4]:
# second, prepare text samples and their labels
print('Processing text dataset')

from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(remove = ('headers', 'footers', 'quotes'))
texts = newsgroups.data
labels = newsgroups.target

print('Found %s texts.' % len(texts))

Processing text dataset
Found 11314 texts.


In [5]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(newsgroups.target_names), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          nb_epoch=10,
          validation_data=(x_val, y_val))

Found 105372 unique tokens.
Shape of data tensor: (11314, 1000)
Shape of label tensor: (11314, 20)
Preparing embedding matrix.
Training model.
Train on 9052 samples, validate on 2262 samples
Epoch 1/10
9052/9052 [==============================] - 12s - loss: 2.7181 - acc: 0.1209 - val_loss: 2.5211 - val_acc: 0.1538
Epoch 2/10
9052/9052 [==============================] - 12s - loss: 2.1078 - acc: 0.2771 - val_loss: 1.8565 - val_acc: 0.3767
Epoch 3/10
9052/9052 [==============================] - 12s - loss: 1.7299 - acc: 0.4070 - val_loss: 1.5981 - val_acc: 0.4399
Epoch 4/10
9052/9052 [==============================] - 12s - loss: 1.5235 - acc: 0.4767 - val_loss: 1.4795 - val_acc: 0.4960
Epoch 5/10
9052/9052 [==============================] - 12s - loss: 1.3549 - acc: 0.5357 - val_loss: 1.4027 - val_acc: 0.5367
Epoch 6/10
9052/9052 [==============================] - 12s - loss: 1.2250 - acc: 0.5872 - val_loss: 1.4537 - val_acc: 0.5340
Epoch 7/10
9052/9052 [==============================]

Compare these results to `loss: 0.3784 - acc: 0.8734 - val_loss: 0.9177 - val_acc: 0.7257`, where the only different was _not_ using the sklearn dataset. We're still training, but there's significant falloff - is this due to fewer texts (`11314` vs `19997`)?

In [6]:
model.fit(x_train, y_train,
          batch_size=128,
          nb_epoch=10,
          validation_data=(x_val, y_val))

Train on 9052 samples, validate on 2262 samples
Epoch 1/10
9052/9052 [==============================] - 12s - loss: 0.7535 - acc: 0.7500 - val_loss: 1.3674 - val_acc: 0.5853
Epoch 2/10
9052/9052 [==============================] - 12s - loss: 0.6864 - acc: 0.7730 - val_loss: 1.5459 - val_acc: 0.5782
Epoch 3/10
9052/9052 [==============================] - 12s - loss: 0.6221 - acc: 0.7990 - val_loss: 1.4108 - val_acc: 0.5973
Epoch 4/10
9052/9052 [==============================] - 12s - loss: 0.5434 - acc: 0.8229 - val_loss: 1.5295 - val_acc: 0.5752
Epoch 5/10
9052/9052 [==============================] - 12s - loss: 0.4930 - acc: 0.8431 - val_loss: 1.7179 - val_acc: 0.5712
Epoch 6/10
9052/9052 [==============================] - 12s - loss: 0.4385 - acc: 0.8649 - val_loss: 1.8488 - val_acc: 0.5853
Epoch 7/10
9052/9052 [==============================] - 12s - loss: 0.4071 - acc: 0.8816 - val_loss: 1.6306 - val_acc: 0.6043
Epoch 8/10
9052/9052 [==============================] - 12s - loss: 0.

Also note that the overfitting issue is present here as well and gets more pronounced with further epochs.